In [1]:
import os
from tqdm import tqdm
import numpy as np
from pathlib import Path
import argparse
import logging
import torch
from torch.utils.data import DataLoader
import wandb
from sklearn.metrics import mean_squared_error
import datasets
from transformers import (Trainer, TrainingArguments, DataCollatorWithPadding,
                          AutoTokenizer, AutoModelForSequenceClassification)

In [2]:
assert torch.cuda.is_available(), 'GPU not found. You should fix this.'

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tiedaar1 (ai-aloe). Use `wandb login --relogin` to force relogin


True

In [4]:
DATA = '../../../data/spancat/'

In [5]:
model_name_or_path = 'microsoft/deberta-v3-large'
dataset_path = DATA+'strategies-ds.hf'
output_dir = DATA+'spancat/results/hp-tuning'
model_max_length = 2056
eval_steps = 1000
eval_accumulation_steps = 2
save_total_limit = 4
batch_size = 16
sweep_id = None
dry_run = False
metric = 'f1'
entity = 'ai-aloe'
project_name = 'metacognitive_classifier'

In [6]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [7]:
def load_dataset(dataset_path):
    ds = datasets.load_from_disk(dataset_path)
    return ds

In [8]:
ds = load_dataset(DATA+'strategies-ds.hf')

In [9]:
def train():
    ''' The main training loop.
    '''
    wandb.init()
    
    config = wandb.config

    model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path,
                                                               num_labels=8,
                                                               problem_type = "multi_label_classification")
        
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        logging_strategy='epoch',
        # eval_steps=eval_steps,
        # save_steps=eval_steps,
        # eval_accumulation_steps=eval_accumulation_steps,
        save_total_limit=save_total_limit,
        optim='adamw_torch',
        learning_rate=config.learning_rate,
        num_train_epochs=config.epochs,
        weight_decay=config.weight_decay,
        warmup_steps = config.warmup_steps,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        disable_tqdm=False,
        report_to='wandb',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    trainer.train()

In [10]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    max_length=model_max_length,
    )

/home/jovyan/conda_envs/wes-env/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16, return_tensors='pt')

# if not sweep_id:
sweep_goal = 'maximize'
    
if dry_run:
    sweep_name = 'dry-run'
else:
    sweep_name = f'{model_name_or_path}'
        
sweep_config = {
    'name': sweep_name,
    'method': 'bayes',
    'metric': {
        'name': f'eval/{metric}',
        'goal': sweep_goal,
    },
    'parameters':
    {
        'epochs': {
            'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        },
        # 'dropout': {
        #     'values': [0, 0.15, 0.3]
        # },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 5e-6,
            'max': 1e-5,
        },
        'weight_decay': {
            'values': [0.01]
        },
        'warmup_steps': {
            'values': [50, 100, 500, 1000]
        },
    },
}

sweep_id = wandb.sweep(sweep_config, entity=entity, project=project_name)

# else:
#     sweep_id = sweep_id
        
wandb.agent(sweep_id, train, count=30)

Create sweep with ID: 8ellv5tx
Sweep URL: https://wandb.ai/ai-aloe/metacognitive_classifier/sweeps/8ellv5tx


wandb: Agent Starting Run: jmnm51f5 with config:
wandb: 	epochs: 7
wandb: 	learning_rate: 8.484830074719205e-06
wandb: 	warmup_steps: 500
wandb: 	weight_decay: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.729600,0.711866,0.518955,0.142466,0.099744,0.249201
2,0.556100,0.382462,0.872951,0.519380,0.660099,0.428115
3,0.363900,0.331549,0.881660,0.492308,0.788732,0.357827
4,0.322600,0.302206,0.894980,0.562900,0.846154,0.421725
5,0.293600,0.279921,0.901639,0.591489,0.885350,0.444089
6,0.258600,0.256210,0.910348,0.665392,0.828571,0.555911
7,0.223500,0.225138,0.924180,0.734767,0.836735,0.654952


eval/accuracy,▁▇▇▇███
eval/f1,▁▅▅▆▆▇█
eval/loss,█▃▃▂▂▁▁
eval/precision,▁▆▇██▇█
eval/recall,▁▄▃▄▄▆█
eval/runtime,▁▅▄▃▅▆█
eval/samples_per_second,█▄▅▆▄▃▁
eval/steps_per_second,█▄▅▆▄▃▁
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/grad_norm,▃▁▂▃▃█▂


...

eval/accuracy,▁▄▇▇██████
eval/f1,▄▅▁▆▆▇▇▇██
eval/loss,██▅▃▂▂▂▁▁▁
eval/precision,▂▃▁▆▇███▇▇
eval/recall,█▇▁▄▄▄▅▅▆▆
eval/runtime,▁▅█▅▃▄▄▄▄▃
eval/samples_per_second,█▄▁▄▆▅▅▅▄▆
eval/steps_per_second,█▄▁▄▆▅▅▅▄▆
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/grad_norm,▃▃▃▁▁█▂▅█▅


wandb: Agent Starting Run: eci9saaa with config:
wandb: 	epochs: 2
wandb: 	learning_rate: 5.067048219163398e-06
wandb: 	warmup_steps: 1000
wandb: 	weight_decay: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.677700,0.671511,0.693135,0.159888,0.142500,0.182109
2,0.666900,0.655243,0.726947,0.043088,0.049180,0.038339


eval/accuracy,▁█
eval/f1,█▁
eval/loss,█▁
eval/precision,█▁
eval/recall,█▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁███
train/global_step,▁▁███
train/grad_norm,▁█
